<a href="https://colab.research.google.com/github/jeongwoo22/ESAA_assignments/blob/main/OB_%EA%B3%BC%EC%A0%9C5_0925.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **앙상블 학습과 랜덤 포레스트**
#### **핸즈온 머신러닝 - Chapter 7**

- 앙상블 : 분류나 회귀 모델 같은 일련의 예측기
- 앙상블 학습 : 예측기 학습
- 앙상블 방법 : 앙상블 학습 알고리즘

- 랜덤 포레스트 : 예측을 하려면, 모든 개별 트리의 예측을 구하면 되는데, 결정 트리의 앙상블 ==> 간단한 방법이지만 오늘날 가장 강력한 머신러닝 알고리즘
- 추가로, 이미 만든 여러 예측기들을 연결하면 더 좋은 예측기가 됨

## **7.1 투표 기반 분류기**

- 예시로 여러 분류기들을 훈련시킴
- 로지스틱 회귀, SVM 분류기, 랜덤 포레스트 분류기 등
- 더 좋은 분류기 만드는 방법 : 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측 하는 것 
- 직접 투표 : 앞처럼 다수결 투표로 정해지는 분류기

- 약한 학습기가 여러개 모여 다양하면, 강한 학습기가 될 수 있다

In [5]:
# 데이서 셋 다운 => 5장에서 사용한 moons 데이터 셋
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
# 여러 분류기를 조합하여 사이킷런의 투표 기반 분류기 만들기 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [7]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


- 간접투표 : 개별 분류기의 예측을 평균내어 확률이 가장 높은 클래스를 예측한다 ==> 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 더 높다

## **7.2 배깅과 페이스팅**

- 배깅 : 훈련 세트에서 중복을 허용하여 샘플링하는 방식
- 페이스팅 : 훈련세트에서 중복을 허용하지 않고 샘플링 하는 방식
- 즉, 배깅과 페이스팅에서는 같은 훈련 샘플을 여러개의 예측기에 걸쳐 사용할 수 있는데, 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있다.

- 통계적 최빈값 : 모든 예측기가 훈련을 마치고, 앙상블이 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만드는 것. 수집함수가 분류일 때

### 7.2.1 사이킷런의 배깅과 페이스팅

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가

In [9]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

0.8986666666666666

In [10]:
bag_clf.oob_decision_function_

array([[0.32275132, 0.67724868],
       [0.34117647, 0.65882353],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.09497207, 0.90502793],
       [0.31147541, 0.68852459],
       [0.01754386, 0.98245614],
       [0.97109827, 0.02890173],
       [0.97765363, 0.02234637],
       [0.74404762, 0.25595238],
       [0.        , 1.        ],
       [0.7173913 , 0.2826087 ],
       [0.85026738, 0.14973262],
       [0.97222222, 0.02777778],
       [0.0625    , 0.9375    ],
       [0.        , 1.        ],
       [0.97837838, 0.02162162],
       [0.94642857, 0.05357143],
       [1.        , 0.        ],
       [0.01704545, 0.98295455],
       [0.39473684, 0.60526316],
       [0.88700565, 0.11299435],
       [1.        , 0.        ],
       [0.97790055, 0.02209945],
       [0.        , 1.        ],
       [0.99428571, 0.00571429],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.62569832, 0.37430168],
       [0.

## **7.3 랜덤 패치와 랜덤 서브스페이스**

- 랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 것
- 랜덤 서브스페이스 방식 : 훈련 샘플을 모두 사용하고 피처는 샘플링하는 것

## **7.4 랜덤 포레스트**

- 랜덤 포레스트 : 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블이다.

In [11]:
from sklearn.ensemble import RandomForestClassifier

# 500개의 트리로 이루어진 랜덤 포레스트 모델
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [12]:
# BaggingClassifier를 사용해 만든 랜덤 포레스트 모델
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### 7.4.1 엑스트라 트리

- 익스트림 랜덤 트리(extremely randomized trees) : 극단적으로 무작위한 트리의 랜덤 포레스트

### 7.4.2 특성 중요도

- feature_importances_ : 사이킷런은 훈련이 끝난 뒤 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결과값을 정규화

In [13]:
from sklearn.datasets import load_iris

# import data
iris = load_iris()

In [14]:
# model fitting
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])

RandomForestClassifier(n_estimators=500, random_state=42)

In [15]:
# print the importances of features
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


## **7.5 부스팅**

### 7.5.1 에이다부스트

- 부스팅(boosting) : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

- 에이다부스트 : 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것 ==> 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 되는데 이게 에이다부스트

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

### 7.5.2 그레이디언트 부스팅

- 그레디언트 부스팅(gradient boosting) : 에이다부스트처럼 앙상블에 이전까지의 오차를 보정하도록 예측 모델을 순차적으로 추가

In [17]:
# model fitting : DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [18]:
y2 = y - tree_reg1.predict(X) # 잔차

# 두 번째 DecisionTreeRegressor 학습
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [19]:
y3 = y - tree_reg2.predict(X) # 잔차

# 세 번째 DecisionTreeRegressor 학습
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [23]:
# import package and split data
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

## **7.6 스태킹**

- 스태킹(stacking) : 앙상블에 속한 모든 예측 모델의 예측을 취합하는 모델

- 메타 학습기(meta learner) 혹은 블렌더(blender)는 각 모델의 예측을 입력 받아 최종 예측을 한다.